# Lab 2

## Bayesian Classifier and Feature Importance

**Note:** Exercises can be autograded and count towards your lab and assignment score. Problems are graded for participation.

In [1]:
from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

In [2]:
%load_ext autoreload
%autoreload 2

# make sure your run the cell above before running this
import Lab2_helper

For this lab, we are going to first implement an empirical naive bayesian classifier, then implement feature importance measures and apply it to a dataset, and finally, we will examine the affect of modifying the priors.

For developing this lab, we can use famous Titanic Kaggle dataset. Description of the data is found https://www.kaggle.com/c/titanic/data.

In [3]:
import pandas as pd
titanic_df = pd.read_csv(
    f"{home}/csc-466-student/data/titanic.csv"
)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We only need a few columns, and I will also perform some preprocessing for you:

In [4]:
features = ['Pclass','Survived','Sex','Age']
titanic_df = titanic_df.loc[:,features]
print('Before')
display(titanic_df.head())
titanic_df.loc[:,'Pclass']=titanic_df['Pclass'].fillna(titanic_df['Pclass'].mode()).astype(int)
titanic_df.loc[:,'Age']=titanic_df['Age'].fillna(titanic_df['Age'].median())
titanic_df.loc[:,'Age']=(titanic_df['Age']/10).astype(str).str[0].astype(int)*10
titranic_df = titanic_df.dropna()
print('After')
titanic_df.head()

Before


,Pclass,Survived,Sex,Age
0,3,0,male,22.0
1,1,1,female,38.0
2,3,1,female,26.0
3,1,1,female,35.0
4,3,0,male,35.0


After


,Pclass,Survived,Sex,Age
0,3,0,male,20
1,1,1,female,30
2,3,1,female,20
3,1,1,female,30
4,3,0,male,30


In [5]:
titanic_df.describe()

,Pclass,Survived,Age
count,891.000000,891.000000,891.000000
mean,2.308642,0.383838,24.208754
std,0.836071,0.486592,13.562886
min,1.000000,0.000000,0.000000
25%,2.000000,0.000000,20.000000
50%,3.000000,0.000000,20.000000
75%,3.000000,1.000000,30.000000
max,3.000000,1.000000,80.000000


#### Problem 1
In your own words, describe the preprocessing steps I took above.

Your solution here.

#### Exercise 1
Create a function to determine the prior probability of ALL the classes in ``y``. The result must be in the form of a Python dictionary such as ``priors = {'Survived=0': 0.4, 'Survived=1': 0.6}``.

In [6]:
survived_priors = Lab2_helper.compute_priors(titanic_df['Survived'])
survived_priors

{'Survived=0': 0.6161616161616161, 'Survived=1': 0.3838383838383838}

In [7]:
Lab2_helper.compute_priors(titanic_df['Age'])

{'Age=0': 0.06958473625140292,
 'Age=10': 0.11447811447811448,
 'Age=20': 0.44556677890011226,
 'Age=30': 0.18742985409652077,
 'Age=40': 0.09988776655443322,
 'Age=50': 0.05387205387205387,
 'Age=60': 0.02132435465768799,
 'Age=70': 0.006734006734006734,
 'Age=80': 0.001122334455667789}

In [8]:
y_example = titanic_df['Age']
y_example.name

'Age'

#### Exercise 2
Create a function to calculate the specific class conditional probability. Assume x and y are pd.Series objects. Assume xv and yv are specific values. This function should return $\Pr(x==xv|y==yv)$.

In [9]:
prob = Lab2_helper.specific_class_conditional(titanic_df['Sex'],'female',titanic_df['Survived'],0)
prob

0.14754098360655737

#### Exercise 3
Now construct a dictionary based data structure that stores all possible class conditional probabilities (e.g., loop through all possible combinations of values). The keys in your dictionary should be of the form "pclass=1|survived=0". ``X`` is a ``pd.DataFrame`` object and ``y`` is a ``pd.Series`` object. You can retrieve the name of the series object ``y`` by accessing ``y.name``.

Aside: I know it might be a bit annoying to store the key of this dictionary as a string instead of as say a tuple of tuples, but I think the way this prints for folks learning this is reason enough to use strings in this instance.

In [10]:
X = titanic_df.drop("Survived",axis=1)
y = titanic_df["Survived"]
probs = Lab2_helper.class_conditional(X,y)
probs

{'Pclass=1|Survived=0': 0.14571948998178508,
 'Pclass=1|Survived=1': 0.39766081871345027,
 'Pclass=2|Survived=0': 0.1766848816029144,
 'Pclass=2|Survived=1': 0.2543859649122807,
 'Pclass=3|Survived=0': 0.6775956284153005,
 'Pclass=3|Survived=1': 0.347953216374269,
 'Sex=female|Survived=0': 0.14754098360655737,
 'Sex=female|Survived=1': 0.6812865497076024,
 'Sex=male|Survived=0': 0.8524590163934426,
 'Sex=male|Survived=1': 0.31871345029239767,
 'Age=0|Survived=0': 0.04371584699453552,
 'Age=0|Survived=1': 0.1111111111111111,
 'Age=10|Survived=0': 0.1111111111111111,
 'Age=10|Survived=1': 0.11988304093567251,
 'Age=20|Survived=0': 0.48816029143898,
 'Age=20|Survived=1': 0.37719298245614036,
 'Age=30|Survived=0': 0.17122040072859745,
 'Age=30|Survived=1': 0.2134502923976608,
 'Age=40|Survived=0': 0.10018214936247723,
 'Age=40|Survived=1': 0.09941520467836257,
 'Age=50|Survived=0': 0.051001821493624776,
 'Age=50|Survived=1': 0.05847953216374269,
 'Age=60|Survived=0': 0.023679417122040074,


#### Exercise 4
Now you are ready to calculate the posterior probabilities for a given sample. Write and test the following function that returns a dictionary where the keys are of the form "Survived=0|Pclass=1,Sex=male,Age=60". Make sure you return 0.5 if the specific combination of values does not exist. ``probs`` and ``priors`` are defined the same as above. ``x`` is a pd.Series object that represents a specific example/sample from our dataset.

In [11]:
probs = Lab2_helper.class_conditional(X,y)
priors = Lab2_helper.compute_priors(y)
x = titanic_df.drop("Survived",axis=1).loc[2]
x

Pclass         3
Sex       female
Age           20
Name: 2, dtype: object

In [12]:
post_probs = Lab2_helper.posteriors(probs,priors,x)
post_probs

{'Survived=0|Pclass=3,Sex=female,Age=20': 0.46699312907215196,
 'Survived=1|Pclass=3,Sex=female,Age=20': 0.533006870927848}

#### Exercise 5
All this is great, but how would you evaluate how we are doing? Create a function called train_test_split that splits our dataframe into a training and testing dataset. To make sure this is done randomly, I have inserted a shuffle into the code for you. The ``test_frac`` is the fraction of the dataset that will be held out for testing.

In [13]:
import numpy as np
np.random.seed(2)
Xtrain,ytrain,Xtest,ytest=Lab2_helper.train_test_split(X,y)
print('Xtrain')
display(Xtrain.head())
print('ytrain')
display(ytrain.head())
print('Xtest')
display(Xtest.head())
print('ytest')
display(ytest.head())

Xtrain


,Pclass,Sex,Age
707,1,male,40
37,3,male,20
615,2,female,20
169,3,male,20
68,3,female,10


ytrain


707    1
37     0
615    1
169    0
68     1
Name: Survived, dtype: int64

Xtest


,Pclass,Sex,Age
716,1,female,30
154,3,male,20
843,3,male,30
24,3,female,0
294,3,male,20


ytest


716    1
154    0
843    0
24     0
294    0
Name: Survived, dtype: int64

#### Exercise 6
For this exercise, create a training dataset of size 50% and then using your solutions to previous exercises, find the prediction accuracy for the test dataset. 

In [14]:
ytest

716    1
154    0
843    0
24     0
294    0
      ..
534    0
584    0
493    0
527    0
168    0
Name: Survived, Length: 445, dtype: int64

In [15]:
np.random.seed(0)
Xtrain,ytrain,Xtest,ytest=Lab2_helper.train_test_split(X,y)
accuracy = Lab2_helper.exercise_6(Xtrain,ytrain,Xtest,ytest)
accuracy

0.7820224719101123

That's not bad!

#### Problem 2:
Is that better than guessing all passengers died? What is the accuracy on the test set if you guessed all passengers died?

Your answer here

**Before proceeding**, make sure you have read the required reading of section 5.5.1 of [this book by Christoph Molnar](https://christophm.github.io/interpretable-ml-book/feature-importance.html). 

#### Excercise 7. Create a function that returns the test based feature importance for our Bayesian classifier.

In [16]:
np.random.seed(0)
Xtrain,ytrain,Xtest,ytest=Lab2_helper.train_test_split(X,y)
importances = Lab2_helper.exercise_7(Xtrain,ytrain,Xtest,ytest)
importances

0.7595505617977528
0.750561797752809
0.7460674157303371
0.7370786516853932
0.755056179775281
0.7640449438202247
0.755056179775281
0.7595505617977528
0.7415730337078652
0.755056179775281
0.5842696629213483
0.604494382022472
0.597752808988764
0.5707865168539326
0.5797752808988764
0.5775280898876405
0.5820224719101124
0.6247191011235955
0.5617977528089888
0.5820224719101124
0.7730337078651686
0.7775280898876404
0.7595505617977528
0.7707865168539326
0.7595505617977528
0.7617977528089888
0.7707865168539326
0.7640449438202247
0.7640449438202247
0.750561797752809


{'Pclass': 0.029662921348314553,
 'Sex': 0.19550561797752802,
 'Age': 0.016853932584269614}

#### Excercise 8. Create a function that returns the train based feature importance for our Bayesian classifier.

In [17]:
np.random.seed(0)
Xtrain,ytrain,Xtest,ytest=Lab2_helper.train_test_split(X,y)
importances = Lab2_helper.exercise_8(Xtrain,ytrain,Xtest,ytest)
importances

{'Pclass': -0.009550561797752854,
 'Sex': 0.1047191011235955,
 'Age': -0.006516853932584321}

#### Problem 3. After you implement this, what is the most important feature? 

Your answer here

#### Problem 4: What are the differences between the two sets of importances?

Your answer here

#### Problem 5: What does a negative value for the importances mean? Consider that it is a very small importance, so what does it say about these features? Consider that question in the context of what you are permuting (training or testing).

Your answer here

Another thing to add to your brain, are there any correlations between features?

In [18]:
Xtrain.corr()

,Pclass,Age
Pclass,1.000000,-0.395434
Age,-0.395434,1.000000


In [19]:
Xtest.corr()

,Pclass,Age
Pclass,1.000000,-0.281029
Age,-0.281029,1.000000


In [20]:
# Good job!
# Don't forget to push with ./submit.sh

#### Having trouble with the test cases and the autograder?

You can always load up the answers for the autograder. The autograder runs your code and compares your answer to the expected answer. I manually review your code, so there is no need to hide this from you.

In [21]:
import joblib
answers = joblib.load(f"{home}/csc-466-student/tests/answers_Lab2.joblib")
answers.keys()

dict_keys(['exercise_1', 'exercise_2', 'exercise_3', 'exercise_4', 'exercise_5', 'exercise_6', 'exercise_7', 'exercise_8'])

In [22]:
answers['exercise_8']

{'Pclass': -0.009550561797752854,
 'Sex': 0.1047191011235955,
 'Age': -0.006516853932584321}